In [1]:
from Dataset.Utils import load_data
import pandas as pd
import numpy as np



In [2]:
data_paths =  {
    "seq_2_terms_df":       "/mnt/d/ML/Kaggle/CAFA6-new/data_packet1/seq_2_terms_H.parquet",
    "train_terms_df":       "/mnt/d/ML/Kaggle/CAFA6/cafa-6-protein-function-prediction/Train/train_terms.tsv",
    "plm_features_path":    "/mnt/d/ML/Kaggle/CAFA6-new/data_packet1/plm_features.npy",
    "prot_2_pmid_path":     "/mnt/d/ML/Kaggle/CAFA6-new/data_packet1/prot_2_pmid.npy",
    "pmid_2_embed_path":    "/mnt/d/ML/Kaggle/CAFA6-new/data_packet1/pmid_2_embed.npy",
    "go_embeds_paths":      "/mnt/d/ML/Kaggle/CAFA6-new/data_packet1/go_embeddings.pkl"
}
data  = load_data(data_paths, aspect='F',  max_terms=128)

Loading PLM features...
Loading BLM data (protein to PMID mapping and PMID embeddings)...
filtering by aspect: F
Padding terms_predicted with random terms from aspect F...


Padding with random F terms:   0%|          | 0/54839 [00:00<?, ?it/s]

After padding - Min: 128, Max: 128, Mean: 128.00
Filtering sequences by term lengths and PLM features availability...
After filtering: 54839 sequences with both PLM features and GO terms


In [4]:
seq_2_terms_df = data['seq_2_terms']
seq_2_terms_df

,qseqid,terms_predicted,terms_true
0,A0A023FBW4,"[GO:0019958, GO:0019956, GO:0019960, GO:001995...","[GO:0019958, GO:0005515, GO:0019956, GO:000548..."
1,A0A023FBW7,"[GO:0005488, GO:0005515, GO:0019955, GO:003575...","[GO:0005515, GO:0019957, GO:0019956, GO:000548..."
2,A0A023FDY8,"[GO:0005488, GO:0005515, GO:0019955, GO:003575...","[GO:0005515, GO:0019957, GO:0019956, GO:000548..."
3,A0A023FF81,"[GO:0019958, GO:0019956, GO:0019960, GO:001995...","[GO:0019958, GO:0005515, GO:0019956, GO:000548..."
4,A0A023FFD0,"[GO:0005488, GO:0005515, GO:0019955, GO:007179...","[GO:0019956, GO:0019955, GO:0005515, GO:001995..."
...,...,...,...
54834,X2JI34,"[GO:0016705, GO:0016830, GO:0052582, GO:003245...","[GO:0016705, GO:0016712, GO:0004497, GO:010622..."
54835,X4Y2L4,"[GO:0016740, GO:0004553, GO:0008474, GO:001978...","[GO:0016787, GO:0016798, GO:0004553, GO:003390..."
54836,X5JA13,"[GO:0070097, GO:1990430, GO:0042562, GO:004698...","[GO:0005515, GO:0005488]"
54837,X5JB51,"[GO:0070097, GO:1990430, GO:0042562, GO:004698...","[GO:0005515, GO:0005488]"


In [5]:
# Check if all true predictions from train_terms_all are present in terms_predicted
def check_coverage(row):
    """Check what percentage of train_terms_all are in terms_predicted"""
    # Check if train_terms_all is None or empty list
    true_key  = 'terms_true'
    if row[true_key] is None or (isinstance(row[true_key], list) and len(row[true_key]) == 0):
        return None
    
    train_set = set(row[true_key])
    pred_set = set(row['terms_predicted'])
    
    # Find terms in train_terms_all that are in terms_predicted
    covered_terms = train_set.intersection(pred_set)
    
    # Calculate coverage percentage
    coverage = len(covered_terms) / len(train_set) * 100 if len(train_set) > 0 else 0
    
    return {
        'total_train_terms': len(train_set),
        'covered_terms': len(covered_terms),
        'missing_terms': len(train_set) - len(covered_terms),
        'coverage_pct': coverage,
        'missing_term_list': list(train_set - covered_terms)
    }

# Apply the check
coverage_results = seq_2_terms_df.apply(check_coverage, axis=1)
coverage_df = pd.DataFrame(coverage_results.tolist())

# Combine with original data
analysis_df = pd.concat([seq_2_terms_df, coverage_df], axis=1)

print("=== Coverage Analysis ===")
print(f"\nOverall Statistics:")
print(f"Average coverage: {coverage_df['coverage_pct'].mean():.2f}%")
print(f"Median coverage: {coverage_df['coverage_pct'].median():.2f}%")
print(f"Min coverage: {coverage_df['coverage_pct'].min():.2f}%")
print(f"Max coverage: {coverage_df['coverage_pct'].max():.2f}%")
print(f"\nRows with 100% coverage: {(coverage_df['coverage_pct'] == 100).sum()} / {len(coverage_df)}")
print(f"Rows with 0% coverage: {(coverage_df['coverage_pct'] == 0).sum()} / {len(coverage_df)}")
print(f"\nAverage missing terms per row: {coverage_df['missing_terms'].mean():.2f}")
print(f"Total train terms checked: {coverage_df['total_train_terms'].sum()}")
print(f"Total covered terms: {coverage_df['covered_terms'].sum()}")
print(f"Total missing terms: {coverage_df['missing_terms'].sum()}")

=== Coverage Analysis ===

Overall Statistics:
Average coverage: 97.85%
Median coverage: 100.00%
Min coverage: 0.00%
Max coverage: 100.00%

Rows with 100% coverage: 51664 / 54839
Rows with 0% coverage: 312 / 54839

Average missing terms per row: 0.13
Total train terms checked: 412277
Total covered terms: 404977
Total missing terms: 7300
